# Introduction to Computer Vision



## Used Libraries

In [24]:
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt

## Camera Module

In [25]:
class Camera(object):
    """ Class for representing pin-hole cameras. """

    def __init__(self, P):
        """ Initialize P = K[R|t] camera model. """
        self.P = P
        self.K = None # calibration matrix
        self.R = None # rotation
        self.t = None # translation
        self.c = None # camera center

    def project(self, X):
        """ Project points in X (4**n array) and normalize coordinates. """
        x = np.dot(self.P, X)
        for i in range(3):
            x[i] /= x[2]

        return x
    

In [26]:
#import camera

# load points
points = np.loadtxt('./Examples/Model_House/house.p3d')
points = np.vstack((points, np.ones(points.shape[1])))
# np.ones(N) returns a array with size N filled with 1s
print(points)

# setup camera
P = np.hstack((np.eye(3), np.array([[0],[0],[-10]])))
# np.eye(N) creates a identity matrix with dimensions NxN

print()
print(P)

cam = Camera(P)
x = cam.project(points)

#plot projection
plt.figure()
plt.plot(x[0], x[1], 'k.')
plt.show()

[[-1.4889    1.09408  -5.89588 ]
 [-1.13932   0.935522 -5.62824 ]
 [ 1.09404   0.738447 -4.53106 ]
 ...
 [-2.51002  -0.144305 -6.7266  ]
 [-1.59536  -1.37243  -3.78376 ]
 [ 1.        1.        1.      ]]

[[  1.   0.   0.   0.]
 [  0.   1.   0.   0.]
 [  0.   0.   1. -10.]]


NameError: name 'dot' is not defined